# Database control on AWS redshift

## Initialize the AWS Service

AWS redshift need to be setup and initialized. Before the initialization, IAM user need to be setup at AWS console. Permission of AWS redshift full access, AWS S3 read-only access. Save the information of IAM account into the ```dwh.cfg```.

In [ ]:
import pandas as pd
import boto3
import json
import configparser

config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

# AWS account info
KEY                     = config.get('AWS','KEY')
SECRET                  = config.get('AWS','SECRET')

# Redshift cluster info
DWH_CLUSTER_TYPE        = config.get('DWH','DWH_CLUSTER_TYPE')
DWH_NUM_NODES           = config.get('DWH','DWH_NUM_NODES')
DWH_NODE_TYPE           = config.get('DWH','DWH_NODE_TYPE')
DWH_CLUSTER_IDENTIFIER  = config.get('DWH','DWH_CLUSTER_IDENTIFIER')
DWH_DB                  = config.get('DWH','DWH_DB')
DWH_DB_USER             = config.get('DWH','DWH_DB_USER')
DWH_DB_PASSWORD         = config.get('DWH','DWH_DB_PASSWORD')
DWH_PORT                = config.get('DWH','DWH_PORT')
DWH_CLUSTER_TYPE        = config.get('DWH','DWH_CLUSTER_TYPE')

# IAM role info
DWH_IAM_ROLE_NAME       = config.get('DWH','DWH_IAM_ROLE_NAME')


### Create IAM role to make Redshift able to read the data from S3 storage

In [ ]:
iam = boto3.client('iam', aws_access_key_id=KEY,
                      aws_secret_access_key=SECRET,
                      region_name='us-east-1')
redshift = boto3.client('redshift', 
                        region_name='us-east-1',
                        aws_access_key_id=KEY,
                        aws_secret_key_access_key=SECRET)

from botocore.exceptions import ClientError

try:
    dwhRole = iam.create_role(
        Path='/',
        RoleName=DWH_IAM_ROLE_NAME,
        Description = "IAM role allow redshift read from S3",
        AssumeRolePolicyDocument=json.dumps(
            {'Statement':[{'Action': 'sts:AssumeRole',
                          'Effect':'Allow',
                          'Principal':{'Service':'redshift.amazon'}}],
             'Version':'2012-10-17'})
    )
except Exception as e:
    print(e)
    
# Attaching policy

iam.attach_role_policy(RoleName=DWH_IAM_ROLE_NAME,
                       PolicyArn=)